In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep
import random
import pandas as pd
import numpy as np
import re
import pickle


In [ ]:
def collect_product_list(type,max_page):
    soup_list = []
    page_count = 1
    while True:
        service_obj = Service("E:/anaconda/Library/bin/chromedriver_win32/chromedriver.exe")
        chrome_options = Options()
        chrome_options.add_argument("--headless") 
        driver = webdriver.Chrome(service=service_obj,options=chrome_options)
        url = 'https://www.paneco.com.sg/t/'+str(type)+'?keywords=&order=&page='+str(page_count)+'&per_page=96&search%5Bdiscount_price%5D%5Bgte%5D=&search%5Bdiscount_price%5D%5Blte%5D='
        driver.get(url)
        sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        soup_list.append(soup)
        print(soup.prettify())
        driver.close()
        
        page_count+=1
        if (page_count == max_page):
            break
    
    return soup_list


In [ ]:
#wine_soup_list = collect_product_list('wine',13)
#beer_soup_list = collect_product_list('beer-and-mixers',3)
sake_soup_list = collect_product_list('liquor/sake',2)

In [ ]:
# Initialized all variables with empty lists
product_id = []
product_name = []
product_link = []
product_price = []
product_alcohol_percentage = []
product_brand = []
product_volume = []
product_country = []
product_varietal = []
product_type = []
product_desc = []
experts_reviews = []
customer_reviews = []
distillery = []
selected_product_link = []
product_name_selected = []

cocktail_recipes = []
cocktail_cuisine = []
cocktail_preparation_time = []
cocktail_nutrition = []
cocktail_servings = []
cocktail_ingredients = []
cocktail_instructions = []
product_count = 0
timer = random.randint(2, 5)

minreq = 15
print(len(sake_soup_list))

In [ ]:
def get_product_link(soup_list, product_df):
    for soup in soup_list:
        list_of_product_var = soup.findAll("div", {'data-hook':'products_list_item'})
        for x in range(len(list_of_product_var)):
            product_id.append(list_of_product_var[x]["id"])
        
        #product_name_var = soup.findAll("h3", {'class':'product__name__text'})
        #for x in range(len(list_of_product_var)):
            #product = list_of_product_var[x].text
            #if product not in product_name:
            #product_name.append(product)
            #product_name = list_of_product[x].text
            #if product_name not in product_name_selected:
                #product_name_selected.append(product_name)
            

        # product price
        product_price_var = soup.findAll("meta", {'itemprop':'lowPrice'})
        for x in range(len(product_price_var)):
            product_price.append(product_price_var[x]["content"])

        #product Links and Names
        product_link_var = soup.findAll("a", {'class':'product__link'})
        for x in range(len(product_link_var)):
            product_item = product_link_var[x]["href"]
            #drop duplicates
            if product_item not in selected_product_link:
                selected_product_link.append(product_item)
                product_name.append(product_link_var[x]["data-label"])
                product_link.append("https://www.paneco.com.sg"+product_item)
    
    i = 0
    for product in product_id:
        product_info = [{
            'product_id' : product_id[i],
            'product_name' : product_name[i],
            'product_price' : product_price[i],
            'product_link' : product_link[i]
        }]
        i+=1

        # append the job posting to dataframe
        temp_df = pd.DataFrame(product_info)
        product_df = pd.concat([product_df, temp_df], axis=0)
    
    return product_df,product_link
        


In [ ]:
print(len(wine_list))

In [ ]:
soup_list = wine_soup_list
wine_list = pd.DataFrame()
wine_list,wine_link_list = get_product_link(wine_soup_list,wine_list)
wine_list.to_csv('wine_product_info_list.csv', encoding='utf-8')

In [ ]:
spirit = pd.DataFrame()

In [ ]:
product_soup_list = []

In [ ]:
def get_link_pages(link_list):
    count = 0
    for link in link_list:
        #if (count > 10):
            #break
        service_obj = Service("E:/anaconda/Library/bin/chromedriver_win32/chromedriver.exe")
        chrome_options = Options()
        chrome_options.add_argument("--headless") 
        driver = webdriver.Chrome(service=service_obj,options=chrome_options)
        driver.get(link)
        sleep(1)
        posting = driver.page_source
        product_soup = BeautifulSoup(posting, 'lxml')
        product_soup_list.append(product_soup)
        print(product_soup.prettify())
        driver.close()
        count+=1
    return product_soup_list

In [ ]:
product_soup_list = get_link_pages(beer_link_list)

In [ ]:

with open('beer_soup_objects.pkl', 'wb') as file:
    pickle.dump(product_soup_list, file)

In [10]:
with open('sake_soup_objects.pkl', 'rb') as file:
    loaded_soup_list = pickle.load(file)

In [11]:
print(len(loaded_soup_list))

34


In [141]:
product_alcohol_percentage = []
product_brand = []
product_volume = []
product_country = []
product_varietal = []
product_type = []
product_desc = []
experts_reviews = []
customer_reviews = []
distillery = []
selected_product_link = []
product_name_selected = []

cocktail_recipes = []
cocktail_cuisine = []
cocktail_preparation_time = []
cocktail_nutrition = []
cocktail_servings = []
cocktail_ingredients = []
cocktail_instructions = []
product_count = 0

In [29]:
print(len(loaded_soup_list))

164


In [12]:
df = pd.DataFrame()
for soup in loaded_soup_list:
    count = 0
    element_names = []
    element_values = []
    property_elements = soup.find_all('td', {'class':'product__properties__name'})
    for element in property_elements:
        value_element = element.find_next_sibling("td", {'class':'product__properties__value'})  
        if value_element:
            element_names.append(element.text)
            element_values.append(value_element.text) 
    product_details = dict(zip(element_names, element_values))
    row = pd.DataFrame([product_details])
    df = pd.concat([df, row], ignore_index=True)
    
df.columns = [re.sub(r'\n\s*', '', col) for col in df.columns]
      

In [13]:
df['Merged Brand'] = df.apply(lambda row: row['Brand'] if pd.notna(row['Brand']) else row['Brands'], axis=1)
df.drop(['Brand', 'Brands'], axis=1, inplace=True)
df.rename(columns={'Merged Brand': 'Brand'}, inplace=True)

In [160]:
df['Merged Brand'] = df.apply(lambda row: row['Finish'] if pd.notna(row['Finish']) else row['Finish.1'], axis=1)
df.drop(['Finish', 'Finish.1'], axis=1, inplace=True)
df.rename(columns={'Merged Brand': 'Finish'}, inplace=True)

In [27]:
df.head()

""
0
1
2
3
4


In [26]:
product_df = pd.read_csv('sake_product_info_list.csv')

In [24]:
merged_df = pd.concat([product_df,df],axis=1)

In [25]:
merged_df.to_csv('sake_product_info_list.csv', encoding='utf-8')

In [18]:
df = pd.DataFrame()
for soup in loaded_soup_list:
    description = soup.find('div', {'class':'product__description'})
    if description:
        description_text = description.text
        #description_texts = description.find('p')
        #description_text = description_texts.text
    expert_review = soup.find('div', {'class':'product__expert-review'})
    if expert_review:
        review_text = ""
        expert_review_text = expert_review.findAll('p')
        for review in expert_review_text:
            review_text += review.text
    else:
        review_text = ""
    customer_reviews = soup.find('div', {'itemprop':'reviewBody'})
    if customer_reviews:
        customer_text = ""
        customer_review_text = customer_reviews.findAll('p')
        for review in customer_review_text:
            customer_text += review.text
    else:
        customer_text = ""
    
    
    row = pd.DataFrame([{'Product_desc':description_text,'Expert Review': review_text, 'Customer Review': customer_text}])
    df = pd.concat([df, row], ignore_index=True)


In [23]:
df = pd.DataFrame()
for soup in loaded_soup_list:
    count = 0
    testing_element_names = []
    testing_element_values = []
    element_names = soup.find_all('span', {'class':'tasting-note-name'})
    if element_names:
        for element in element_names:
            testing_element_names.append(element.text)
            element_values = element.find_parent("div",{'class':'wrapper text-center'}).find_next_sibling("div", {'class':'tasting-note-description'})
            testing_element_values.append(element_values.text)
    product_details = dict(zip(testing_element_names, testing_element_values))
    row = pd.DataFrame([product_details])
    df = pd.concat([df, row], ignore_index=True)
    


In [110]:
df = pd.DataFrame()
for soup in loaded_soup_list:
    score = soup.find('div', {'class':'award-icon points'})
    if score:
        score_text = score.text
    else:
        score_text = ""
    row = pd.DataFrame([{'Score':score_text}])
    df = pd.concat([df, row], ignore_index=True)

In [205]:
df.head()

,Food Pairing
0,[]
1,"[Beef, Chicken, Seafood, Pork]"
2,"[Fish, Seafood, Salad, Salmon]"
3,"[Chicken, Fish, Cheese, Dessert]"
4,"[Beef, Pork, Pasta, Lamb]"


In [204]:
df.count()

Food Pairing    1110
dtype: int64

In [203]:
df = pd.DataFrame(columns=['Food Pairing'])

for soup in loaded_soup_list:
    food_list = []
    foods = soup.find_all('div', {'class': 'food-name'})  
    if foods:
        for food in foods:
            food_list.append(food.text)
    
    series = pd.Series({'Food Pairing': food_list})
    df = df.append(series, ignore_index=True)



C:\Users\20571\AppData\Local\Temp\ipykernel_8424\1961694760.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(series, ignore_index=True)
C:\Users\20571\AppData\Local\Temp\ipykernel_8424\1961694760.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(series, ignore_index=True)
C:\Users\20571\AppData\Local\Temp\ipykernel_8424\1961694760.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(series, ignore_index=True)
C:\Users\20571\AppData\Local\Temp\ipykernel_8424\1961694760.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(series, ignore_index=True)
C:\Users\20571\AppData\L

In [28]:
beer = pd.read_csv('beer_product_info_list.csv').assign(alcohol_type='Beer')
sake = pd.read_csv('sake_product_info_list.csv').assign(alcohol_type='Sake')
wine = pd.read_csv('wine_product_info_list.csv').assign(alcohol_type='Wine')
spirit = pd.read_csv('spirit_product_info_list.csv').assign(alcohol_type='Spirit')

merged_df = pd.concat([beer,sake,wine,spirit],axis=0)

In [35]:
merged_df.to_csv('merged_product_info_list.csv', encoding='utf-8')

In [30]:
merged_df = pd.read_csv('merged_product_info_list.csv')

In [31]:
merged_df.drop(columns={'Unnamed: 0.5','Unnamed: 0.6','Unnamed: 0.7','Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'},inplace=True)

In [33]:
merged_df.drop(columns={'Product_desc.1','Expert Review.1','Customer Review.1'})

,product_id,product_name,product_price,product_link,Product_desc,Expert Review,Customer Review,Appearance,Nose,Palate,...,Brand,alcohol_type,Varietals,Organic,Vintage,Style,palate,Score,Finish,Type
0,product_3430,12-x-erdinger-dunkel-beer-bottle-500ml,59.95,https://www.paneco.com.sg/products/12-x-erding...,Guillaume and Aude Lavollée represent the four...,NaN,NaN,Golden yellow,"Pear, peach, citrus and acacia notes","Friendly, full-bodied and fresh with good aci...",...,Erdinger,Beer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,product_505,12-x-erdinger-weissbier-beer-bottle-500ml,58.50,https://www.paneco.com.sg/products/12-x-erding...,12 Bottle Case of fine Baverian Beer. Every si...,“ERDINGER Weissbier with fine yeast is the und...,Definitely makes it to number #1 on my list of...,NaN,NaN,NaN,...,Erdinger,Beer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,product_3197,20-x-franziskaner-weissbier-bottle-case-500ml,94.00,https://www.paneco.com.sg/products/20-x-franzi...,Franziskaner Weissbier beer bottle (20 case) C...,NaN,NaN,NaN,NaN,NaN,...,Franziskaner,Beer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,product_823,20-x-paulaner-original-munchner-hell-beer-bott...,98.50,https://www.paneco.com.sg/products/20-x-paulan...,A true Lager beer needs four ingredients: Wate...,“A true classic of essential Munich and Bavari...,The Original Hell is probably the least apprec...,NaN,NaN,NaN,...,Paulaner,Beer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,product_497,20-x-paulaner-hefe-weissbier-beer-bottle-case-...,93.42,https://www.paneco.com.sg/products/20-x-paulan...,Naturally cloudy and shining silky gold in the...,"“For friends of good beer, this is one of the ...",NaN,NaN,NaN,NaN,...,Paulaner,Beer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3432,product_685,wyborowa-vodka-750ml,46.50,https://www.paneco.com.sg/products/wyborowa-vo...,\n The Pure Rye Grain combine...,"¡°Wyborowa has a grainy, fruity and slightly f...",NaN,NaN,NaN,NaN,...,Wyborowa,Spirit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3433,product_3719,x-by-glenmorangie-single-malt-700ml,86.74,https://www.paneco.com.sg/products/x-by-glenmo...,\nGlenmorangie X is a fab single malt from the...,NaN,NaN,NaN,NaN,NaN,...,Glenmorangie,Spirit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3434,product_3428,yushan-signature-sherry-cask-single-malt-whisk...,139.95,https://www.paneco.com.sg/products/yushan-sign...,\nYushan Signature Sherry Cask single malt has...,NaN,NaN,NaN,NaN,NaN,...,Yushan,Spirit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3435,product_2006,zacapa-centenario-23-year-1l-w-gift-box,135.50,https://www.paneco.com.sg/products/zacapa-cent...,\nWonderfully intricate with honeyed buttersco...,NaN,NaN,NaN,NaN,NaN,...,Zacapa,Spirit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
